In [14]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import math
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import PIL
import matplotlib.pyplot as plt
import glob
import warnings
warnings.filterwarnings('ignore')
import tensorflow_addons as tfa
from tensorflow.keras.metrics import top_k_categorical_accuracy, categorical_accuracy

In [15]:
root = "D:\IIIT-D\Summer Semester\CAPSTONE PROJECT\Complete Dataset"

In [16]:
train_dir = root+'/Hair Removed Images/train'
val_dir = root+'/Hair Removed Images/val'
test_dir = root+'/Hair Removed Images/test'
datagen = ImageDataGenerator(preprocessing_function = tf.keras.applications.mobilenet.preprocess_input)

# test_datagen = ImageDataGenerator(rescale=1./255)

train_data = datagen.flow_from_directory(train_dir,target_size=(224,224),batch_size=4,class_mode='categorical',seed=42)

val_data = datagen.flow_from_directory(val_dir,target_size=(224,224),batch_size=4,class_mode='categorical',seed=42)

test_data = datagen.flow_from_directory(test_dir,target_size=(224,224),batch_size=4,class_mode='categorical',shuffle=False)

Found 40253 images belonging to 7 classes.
Found 1000 images belonging to 7 classes.
Found 1006 images belonging to 7 classes.


In [17]:
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

num_labels = 7

num_labels = 7

base_model = ResNet101(include_top=False, input_shape=(128, 128, 3),pooling = 'avg', weights = 'imagenet')
model = tf.keras.models.Sequential()
model.add(base_model)
model.add(Dropout(0.5))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(num_labels, activation = 'softmax',))

In [18]:
def top_2_acc(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=2)

def top_3_acc(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

lrReduction=ReduceLROnPlateau(monitor='val_accuracy',
                              factor=0.5,
                              patience=2,
                              verbose=1,
                              mode='auto',
                              min_lr=0.00001)

early_stopping = EarlyStopping(monitor="val_accuracy", patience=5, verbose=1)

In [19]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy',
                                                                        top_2_acc,
                                                                        top_3_acc,
                                                                        tf.metrics.AUC(curve='ROC'),
                                                                        tfa.metrics.MatthewsCorrelationCoefficient(num_classes=7)])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet101 (Functional)       (None, 2048)              42658176  
_________________________________________________________________
dropout_4 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               262272    
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 7)                 903       
Total params: 42,921,351
Trainable params: 42,816,007
Non-trainable params: 105,344
_________________________________________________________________


In [20]:
step_size_train=train_data.n//train_data.batch_size
history = model.fit_generator(generator=train_data,
                   steps_per_epoch=step_size_train,
                   validation_data=val_data,
                    epochs=50, 
                    verbose = 1,
                    callbacks=[lrReduction,early_stopping])

Epoch 1/50
10063/10063 [==============================] - 1742s 172ms/step - loss: 1.9181 - accuracy: 0.1849 - top_2_acc: 0.3405 - top_3_acc: 0.4888 - auc_2: 0.5668 - MatthewsCorrelationCoefficient: 0.0486 - val_loss: 2.0236 - val_accuracy: 0.0480 - val_top_2_acc: 0.1310 - val_top_3_acc: 0.2330 - val_auc_2: 0.4311 - val_MatthewsCorrelationCoefficient: -0.0070
Epoch 2/50
10063/10063 [==============================] - 1729s 172ms/step - loss: 1.7536 - accuracy: 0.2837 - top_2_acc: 0.4592 - top_3_acc: 0.6092 - auc_2: 0.6832 - MatthewsCorrelationCoefficient: 0.1685 - val_loss: 1.6170 - val_accuracy: 0.4080 - val_top_2_acc: 0.4870 - val_top_3_acc: 0.6030 - val_auc_2: 0.7270 - val_MatthewsCorrelationCoefficient: 0.1880
Epoch 3/50
10063/10063 [==============================] - 1730s 172ms/step - loss: 1.7472 - accuracy: 0.2910 - top_2_acc: 0.4685 - top_3_acc: 0.6173 - auc_2: 0.6893 - MatthewsCorrelationCoefficient: 0.1774 - val_loss: 1.3249 - val_accuracy: 0.5370 - val_top_2_acc: 0.6510 - val

In [21]:
loss,accuracy,top_2,top_3,auc,mcc = model.evaluate(train_data)

10064/10064 [==============================] - 527s 52ms/step - loss: 240.5943 - accuracy: 0.7210 - top_2_acc: 0.9217 - top_3_acc: 0.9724 - auc_2: 0.8995 - MatthewsCorrelationCoefficient: 0.6797


In [22]:
loss,accuracy,top_2,top_3,auc,mcc = model.evaluate(val_data)

250/250 [==============================] - 13s 52ms/step - loss: 146.2075 - accuracy: 0.7010 - top_2_acc: 0.8910 - top_3_acc: 0.9700 - auc_2: 0.9174 - MatthewsCorrelationCoefficient: 0.4346


In [23]:
loss,accuracy,top_2,top_3,auc,mcc = model.evaluate(test_data)

252/252 [==============================] - 14s 54ms/step - loss: 144.7224 - accuracy: 0.6889 - top_2_acc: 0.8718 - top_3_acc: 0.9453 - auc_2: 0.9106 - MatthewsCorrelationCoefficient: 0.4226


In [24]:
from sklearn import metrics

true_test_labels=test_data.labels

true_test_labels=np.array(true_test_labels)

np.unique(true_test_labels,return_counts=True)

pred_test_labels=model.predict(test_data)

pred_test_labels=np.argmax(pred_test_labels,axis=1)

metrics.accuracy_score(true_test_labels,pred_test_labels)

report = metrics.classification_report(true_test_labels,pred_test_labels)
print(report)

              precision    recall  f1-score   support

           0       0.56      0.44      0.49        34
           1       0.44      0.54      0.49        52
           2       0.43      0.48      0.45       111
           3       0.33      0.33      0.33        12
           4       0.50      0.23      0.32       112
           5       0.84      0.83      0.83       670
           6       0.17      0.73      0.28        15

    accuracy                           0.69      1006
   macro avg       0.47      0.51      0.46      1006
weighted avg       0.71      0.69      0.69      1006



In [26]:
# model.save("D:\IIIT-D\Summer Semester\CAPSTONE PROJECT\Saved Models\Resnet_101.h5")